In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import dask
import dask.dataframe as dd
import dask.array as da
import dask.distributed
import numpy as np
from sklearn.preprocessing import MinMaxScaler

random.seed(20816609)

In [ ]:
data = np.load('Xy_train.npz')
X_train = data['X']
y_train = data['y']
X_test = np.load('X_test.npz')
X_test = X_test['X']

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train
                                                                , test_size=0.3, random_state=42)

In [ ]:
# Scale the data to (0, 1)
min_value = min(y_train)
max_value = max(y_train)
y_train = (y_train - min_value) / (max_value - min_value)

min_value = min(y_validation)
max_value = max(y_validation)
y_validation = (y_validation - min_value) / (max_value - min_value)

In [ ]:
# create model instance
model = xgb.XGBRegressor(
    objective='reg:logistic',
    eval_metric='rmse',
    early_stopping_rounds=15,
    n_estimators=2250,
    learning_rate=0.15,
    device='cuda',
    tree_method='hist',
    alpha=35,
    reg_lambda=30
)

# Train the model
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_validation, y_validation)])

original_labels = np.array([0, 1, 2, 3, 4, 5, 6, 7])
scaled_labels = np.unique(y_train)

# Initialize MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(original_labels.reshape(-1, 1))

predictions_scaled = model.predict(X_test)
predictions_original_scale = scaler.inverse_transform(predictions_scaled.reshape(-1, 1))
predictions_original_scale = np.round(predictions_original_scale).astype(int)

df = pd.DataFrame({
    'ID': np.arange(0, 100000),
    'Predicted': predictions_original_scale.ravel()
})
df.set_index('ID', inplace=True)
df.to_csv('submission.csv')